In [ ]:
import pandas as pd
import dask.dataframe as dd
from scipy import spatial
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import numpy as np
import dask.array as da

In [ ]:
try:
    mortage_data=pd.read_csv("/Users/rashmi/Documents/RA/for Rashmi/mortgage_deeds_10.csv")
except:
    mortage_data=pd.read_csv("./mortgage_deeds_10.csv")
# mortage_data.head()
seller_info=mortage_data[["record_number","sellerlastname","sellerfirstname","sellername1","sellername2"]]
seller_info=seller_info[~seller_info["sellername1"].str.contains("COUNTY",na=False)]
print(seller_info.shape)
seller_info=seller_info[~((seller_info["sellername1"].str.endswith(" LLC",na=False)) &
                         (seller_info["sellername2"].isna()))]
seller_info=seller_info[~((seller_info["sellername1"].str.endswith(" TRUST ",na=False)) &
                         (seller_info["sellername2"].isna()))]
seller_info=seller_info[~((seller_info["sellername1"].str.endswith(" MGMT",na=False)) &
                         (seller_info["sellername2"].isna()))]
seller_info=seller_info[~((seller_info["sellername1"].str.endswith(" TRUST 2",na=False)) &
                         (seller_info["sellername2"].isna()))]
seller_info=seller_info[~seller_info["sellername1"].isna()]
def get_split_on_and(s1,s2):
    s0=s1.split("&")[0]
    #add scenario of keepnig the name after & in seller name2
    return s0
seller_info["sellername1"]=seller_info.apply(lambda x:get_split_on_and(x["sellername1"],x["sellername2"]),axis=1)
print(seller_info.shape)
seller_info.head()

In [ ]:
try:
    manager_data=pd.read_excel("/Users/rashmi/Documents/RA/for Rashmi/LexisNexis_Managers.xlsx",sheet_name="data",
engine="openpyxl")
except:
    manager_data=pd.read_excel("./LexisNexis_Managers.xlsx",sheet_name="data",engine="openpyxl")
    
print(manager_data.shape)
manager_data=manager_data[~(manager_data["managername"].str.endswith(" LLC",na=False))]
manager_data=manager_data[~(manager_data["managername"].str.endswith(" TRUST ",na=False))]
manager_data=manager_data[~(manager_data["managername"].str.endswith(" MGMT",na=False))]
manager_data=manager_data[~manager_data["managername"].isna()]
print(manager_data.shape)
manager_data.head()

In [ ]:
def get_name(x,y,type):
    str_split_arr=str(x).rsplit(" ",1)
    length=len(str_split_arr)
    if y==1 and type=="last"and length==1:
        return str_split_arr[0]
    elif y==1 and type=="last"and length>1:
        middle_name=str_split_arr[0]
        return str_split_arr[1]
    elif y==0 and type=="first":
        return str_split_arr[0]
    elif y==1 and type=="first":
        return ""
    elif y==0 and type=="last" and length>1:
        middle_name=str_split_arr[0]
        return str_split_arr[1]
    elif y==0 and type=="last" and length==1:
        return str_split_arr[0]
    else:
        return ""
    
def get_middle_name(x,z,y):
    str_split_arr=str(x).rsplit(" ",1)
    split_firstname=str(z).rsplit(" ",1)
    first_length=len(split_firstname)
    str_length=len(str_split_arr)
    if y==1 and str_length>1:
        middle_name=str_split_arr[0]
    elif y==0 and str_length>1:
        if first_length>1:
            middle_name=split_firstname[1]
        else:
            middle_name=""
    else:
        middle_name=""
    
    return middle_name

    
def update_firstname(str1,str2):
    if str1.endswith(str2):
        str1=str1.replace(str2,"")
    str1=str1.strip()
    return str1
    
    
manager_data["managerfirstname"]=manager_data.apply(lambda x :get_name(x["managername"],
                                                                        x["lastname_only"],"first"),axis=1)
manager_data["managerlastname"]=manager_data.apply(lambda x :get_name(x["managername"],
                                                                       x["lastname_only"],"last"),axis=1)
manager_data["managermiddlename"]=manager_data.apply(lambda x :get_middle_name(x["managername"],x["managerfirstname"],
                                                                       x["lastname_only"]),axis=1)
manager_data["managerfirstname"]=manager_data.apply(lambda x :update_firstname(x["managerfirstname"],
                                                                               x["managermiddlename"]),axis=1)
manager_data.head()

In [ ]:
# corpus = api.load('text8')
model= api.load("glove-wiki-gigaword-100")
# model = Word2Vec(corpus)

In [ ]:
s0 = 'Mark zuckerberg'
s1 = 'Mark zuckerberg'
s2 = 'zuckerberg mark'
s3 = 'zuckerberg v mark'
s4="mmmmm"
def preprocess(s):
    return [i.lower() for i in str(s).split()]
def get_vector(s):
     return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0) 
    
print(spatial.distance.cosine(get_vector(s1), get_vector(s0)))
print(spatial.distance.cosine(get_vector(s2), get_vector(s0)))
print(spatial.distance.cosine(get_vector(s3), get_vector(s0)))
print(spatial.distance.cosine(get_vector(s4), get_vector(s0)))

In [ ]:
unknown_names=[]

def preprocess(s):
    return [i.lower() for i in str(s).split()]

# empty_vector=da.from_array(np.zeros(100,), chunks=((100, 0)))
  
def get_vector(x):
    try:
        sum_vec=np.sum(np.array([model[i] for i in preprocess(x)]), axis=0)
        return sum_vec
    
    except:
        return np.zeros(100,)
    
def find(x):
    print(x)

seller_info["sellername1_vector"]=seller_info["sellername1"].apply(lambda x: get_vector(x))
seller_info["Is_empty_vector"]=seller_info["sellername1_vector"].apply(lambda x : 1 
                                                                         if (x==empty_vector).all() else 0)
v_seller_info=seller_info[seller_info["Is_empty_vector"]==0]
v_empty_seller_info=seller_info[seller_info["Is_empty_vector"]==1]

In [ ]:
v_empty_seller_info["sellername1"].head()

In [ ]:
def preprocess(s):
    return [i.lower() for i in str(s).split()]

def get_vector(x):
    try:
        return np.sum(np.array([model[i] for i in preprocess(x)]), axis=0)
    except:
        return np.zeros(100,)

manager_data["managername_vector"]=manager_data["managername"].apply(lambda x: get_vector(x))
manager_data["Is_empty_vector"]=manager_data["managername_vector"].apply(lambda x : 1 if (x==empty_vector).all() else 0)
v_manager_data=manager_data[manager_data["Is_empty_vector"]==0]
print(v_manager_data.shape,manager_data.shape)

In [ ]:
v_manager_data.dtypes

In [ ]:
import pandas as pd
import dask.dataframe as dd
df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                   'y': [1., 2., 3., 4., 5.]})
ddf = dd.from_pandas(df, npartitions=2)
res = ddf.x.map_partitions(lambda x: len(x))
print(res)
ddf["res"]=res
ddf.compute()

In [70]:
import dask
v_dseller_info = dd.from_pandas(v_seller_info, npartitions=10)
print(v_dseller_info.dtypes)
v_dseller_info.head()

record_number         object
sellerlastname        object
sellerfirstname       object
sellername1           object
sellername2           object
sellername1_vector    object
Is_empty_vector        int64
dtype: object


,record_number,sellerlastname,sellerfirstname,sellername1,sellername2,sellername1_vector,Is_empty_vector
7,8S10,NaN,NaN,ATLANTIC HM LOANS,NaN,"[-0.15005797, -0.219821, 1.1671159, 0.2612043,...",0
11,12S10,BOULDEN,BRIAN,BOULDEN BRIAN J,NaN,"[-0.56598, -0.43407, -0.77338994, -1.30842, -1...",0
14,15S10,NaN,NaN,SHALLCROSS MTG,NaN,"[-0.053380013, -0.91077006, -0.31649002, 0.474...",0
15,16S10,NaN,NaN,LENAPE DEV INC,NaN,"[1.24189, -0.790416, 0.94798994, -0.56247, -0....",0
18,19S10,NaN,NaN,NVR INC,NaN,"[0.78047, -0.379701, 0.298482, -0.34065, 1.232...",0


In [ ]:

# print(v_dseller_info.npartitions,v_dseller_info.dtypes,v_dseller_info.divisions)
# print(v_dseller_info.compute().shape,seller_info.shape,v_empty_seller_info.shape)
# print(v_dseller_info.dtypes)
# v_dseller_info.head()

def simple_map(df,s0):
    df['Is_empty_vector'] = df['sellername1_vector'].map(lambda x:spatial.distance.cosine(x,s0))
    return df

def get_cosine_distance(s0):
    print(v_dseller_info.compute().shape)
    v_dseller_info_new=v_dseller_info.map_partitions(simple_map,s0, meta=seller_info[:0])
    dseller_info_match=v_dseller_info_new[v_dseller_info_new["Is_empty_vector"]<0.2]
    print(dseller_info_match.shape)
    dseller_info_match=dseller_info_match.compute()
    print("fdfd")
    if dseller_info_match.shape[0]==1:
        rec_num=dseller_info_match['record_number'].values[0]
        return rec_num
    elif dseller_info_match.shape[0]>1:
        rec_num=dseller_info_match['record_number'].values
        return rec_num
    else:
        return "no_match"
                                                                                                                                           

v_manager_data["Iteration2"]=v_manager_data["managername_vector"].apply(lambda x : get_cosine_distance(x))

print(v_manager_data.shape)
print(v_manager_data[v_manager_data["Iteration2"]=="multiple"].shape)
print(v_manager_data[v_manager_data["Iteration2"]!="no_match"].shape)
print(v_manager_data[v_manager_data["Iteration2"]=="no_match"].shape)


(381004, 7)
(Delayed('int-b06b1e07-f2e1-4b9d-ae24-190530fe4442'), 7)


/var/folders/d9/pk8y025s0x79tw6tvd9ckcm00000gn/T/ipykernel_6957/2086648068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_empty_vector'] = df['sellername1_vector'].map(lambda x:spatial.distance.cosine(x,s0))


fdfd
(381004, 7)
(Delayed('int-65fbbe7c-8233-4944-97b7-2f13d553d7e1'), 7)


/var/folders/d9/pk8y025s0x79tw6tvd9ckcm00000gn/T/ipykernel_6957/2086648068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_empty_vector'] = df['sellername1_vector'].map(lambda x:spatial.distance.cosine(x,s0))


fdfd
(381004, 7)
(Delayed('int-634987fd-ac58-4c1d-8f47-2cd0dd651cec'), 7)


...

/var/folders/d9/pk8y025s0x79tw6tvd9ckcm00000gn/T/ipykernel_6957/2086648068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_empty_vector'] = df['sellername1_vector'].map(lambda x:spatial.distance.cosine(x,s0))


fdfd
(381004, 7)
(Delayed('int-eea06e29-594f-4403-80b7-b3487859e66f'), 7)


/var/folders/d9/pk8y025s0x79tw6tvd9ckcm00000gn/T/ipykernel_6957/2086648068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_empty_vector'] = df['sellername1_vector'].map(lambda x:spatial.distance.cosine(x,s0))


fdfd
(381004, 7)
(Delayed('int-448aa6dc-8c98-4ab0-881a-cdfb05a56a88'), 7)


/var/folders/d9/pk8y025s0x79tw6tvd9ckcm00000gn/T/ipykernel_6957/2086648068.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_empty_vector'] = df['sellername1_vector'].map(lambda x:spatial.distance.cosine(x,s0))


In [68]:
# df = pd.DataFrame({'A':['ant','ant','cherry', 'bee', 'ant'], 'B':['cat','peach', 'cat', 'cat', 'peach'], 'C':['dog','dog','roo', 'emu', 'emu']})
# ddf = dd.from_pandas(df, npartitions=2)
# vc = ddf.A.value_counts().compute()
# vc /= vc.sum()  # because dask's value_count doesn't normalise

# def simple_map(df):
#     df['A'] = df['A'].map(lambda x: x if vc[x] > 0.5 else 'other')
#     return df

# a=ddf.map_partitions(simple_map, meta=df[:0])
# a.compute()
seller_info[:0]

,record_number,sellerlastname,sellerfirstname,sellername1,sellername2,sellername1_vector,Is_empty_vector


In [ ]:
import datetime
print(datetime.datetime.now())